In [ ]:
import argparse
import os
import sys
sys.path.append('/users/vesalaia/pipelinv2')
sys.path.append("/users/vesalaia/.local/lib/python3.9/site-packages/bin")
sys.path.append("/users/vesalaia/.local/lib/python3.9/site-packages/lib/python3.9/site-packages")

In [ ]:
import logging

import datetime
now = datetime.datetime.now()

timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")
log_file_name = f"table_recognition_{timestamp}.log"

logging.basicConfig(
    filename=log_file_name,
    level=logging.INFO,
    format='[%(asctime)s] %(levelname)s - %(message)s',
    datefmt='%H:%M:%S'
)


In [ ]:
from config.options import Options

from data.dataset import OCRDatasetInstanceSeg
from pipeline.engine import initFolder, extractText, pipelineTask
from utils.config_check import configuration_ok

In [ ]:
def str2bool(v):
    if isinstance(v, bool):
        return v
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        logging.error('A serious error occurred. Boolean value expected.')
        raise argparse.ArgumentTypeError('Boolean value expected.')

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def crop_polygon_from_image(image, polygon):
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, [polygon], (255, 255, 255))
    cropped_image = cv2.bitwise_and(image, mask)
    x, y, w, h = cv2.boundingRect(polygon)
    cropped_image = cropped_image[y:y+h, x:x+w]
    return cropped_image


In [ ]:
import cv2
import numpy as np
import random

# Function to generate n random colors
def generate_random_colors(n_colors=20):
    random.seed(42)  # Set a seed for reproducibility (optional)
    colors = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for _ in range(n_colors)]
    return colors


def draw_polygons_on_image(image, polygons):
    # Copy the original image to avoid modifying it
    output_image = image.copy()

    # Define a list of random colors (you can have fewer colors than polygons)
    colors = generate_random_colors(20)

    # Loop over each polygon and draw it on the image
    for i, polygon in enumerate(polygons):
        # Select a color from the list (cycling through the list if needed)
        color = colors[i % len(colors)]

        # Draw the polygon
        cv2.polylines(output_image, [polygon], isClosed=True, color=color, thickness=2)

        # Fill the polygon with some transparency
        overlay = output_image.copy()
        cv2.fillPoly(overlay, [polygon], color=color)
        alpha = 0.2  # Transparency factor
        cv2.addWeighted(overlay, alpha, output_image, 1 - alpha, 0, output_image)

    return output_image



In [ ]:
"""
Tasks:
    init:
    region:
    line:
    table:
    recognize:
    update:
    text:
    json:
"""

In [ ]:
def processTask(cfgfile, executeTask, infolder, inpage, outpage):
    logging.info(f"Configuration file: {cfgfile}")
    if cfgfile != None:
        opts =  Options(cfgfile)

    if configuration_ok(opts, executeTask.lower()):    
        if executeTask.lower() in ["init", "i"]:
            executeTask = "init"
            if outpage == "":
                outpage = "page"
            if infolder != "":
                logging.info(f"Task:{executeTask} Folder:{infolder} XML-output: {outpage}")
                initFolder(opts, infolder, outpage) 
        elif executeTask.lower() in ["text", "json", "csv"]:
            executeTask = executeTask.lower()
            if inpage == "":
                inpage = "pageText"
            if outpage == "":
                outpage = "text"
            if infolder != "":
                logging.info(f"Task:{executeTask} Folder:{infolder} XML-input:{inpage} XML-output: {outpage}")
                extractText(opts, infolder, inpage, outpage)
        elif executeTask.lower() in ["detectregion", "dr", "region"]:
            executeTask = "region"
            if inpage == "":
                inpage = "page"
            if outpage == "":
                outpage = "pageRD"
            if infolder != "":
                RO_groups = opts.RO_region_groups
                logging.info(f"Task:{executeTask} Folder:{infolder} XML-input:{inpage} XML-output: {outpage}")
                pipelineTask(opts, executeTask, infolder, inpage=inpage, outpage=outpage, tryMerge=tryMerge, 
                             reading_order=reading_order)
        elif executeTask.lower() in ["detectlines", "dl", "line"]:
            executeTask = "line"
            if inpage == "":
                inpage = "pageRD"
            if outpage == "":
                outpage = "pageLD"
            if infolder != "":
                RO_groups = opts.RO_region_groups
                one_page_per_image = True
                logging.info(f"Task:{executeTask} Folder:{infolder} XML-input:{inpage} XML-output: {outpage}")
                pipelineTask(opts, executeTask, infolder, inpage=inpage, outpage=outpage, tryMerge=tryMerge,
                            reading_order=reading_order, line_model="mask r-cnn")
        elif executeTask.lower() in ["recognizetext", "rt", "recognize"]:
            executeTask = "recognize"
            if inpage == "":
                inpage = "pageLD"
            if outpage == "":
                outpage = "pageText"
            if infolder != None:
                RO_groups = opts.RO_region_groups
                logging.info(f"Task:{executeTask} Folder:{infolder} XML-input:{inpage} XML-output: {outpage}")
                pipelineTask(opts, executeTask, infolder, inpage=inpage, outpage=outpage, 
                             reading_order=reading_order)
        elif executeTask.lower() in ["update", "u"]:
            executeTask = "update"
            if inpage == "":
                inpage = "pageLD"
            if outpage == "":
                outpage = "pageU"
                if infolder != "":
                    RO_groups = opts.RO_line_groups
                    logging.info(f"Task:{executeTask} Folder:{infolder} XML-input:{inpage} XML-output: {outpage}")
                    pipelineTask(opts, executeTask, infolder, inpage=inpage, outpage=outpage, 
                             reading_order=reading_order, combine=combine)
        elif executeTask.lower() in ["table", "t"]:
            executeTask = "table"
            if inpage == "":
                inpage = "page"
            if outpage == "":
                outpage = "pageTbl"
            if infolder != "":
                logging.info(f"Task:{executeTask} Folder:{infolder} XML-input:{inpage} XML-output: {outpage}")
                pipelineTask(opts, executeTask, infolder, inpage=inpage, outpage=outpage, 
                             reading_order=False, combine=False)
        elif executeTask.lower() in ["cell", "c"]:
            executeTask = "cell"
            if inpage == "":
                inpage = "pageTbl"
            if outpage == "":
                outpage = "pageCell"
            if infolder != "":
                logging.info(f"Task:{executeTask} Folder:{infolder} XML-input:{inpage} XML-output: {outpage}")
                pipelineTask(opts, executeTask, infolder, inpage=inpage, outpage=outpage, 
                             reading_order=False, combine=False)
        elif executeTask.lower() in ["cellrecognize", "cr"]:
            executeTask = "cellrecognize"
            if inpage == "":
                inpage = "pageCell"
            if outpage == "":
                outpage = "pageText"
            if infolder != None:
                logging.info(f"Task:{executeTask} Folder:{infolder} XML-input:{inpage} XML-output: {outpage}")
                pipelineTask(opts, executeTask, infolder, inpage=inpage, outpage=outpage)
        else:
            logging.error(f"Task not recognized: {executeTask}")


In [ ]:
cfgfile = "/users/vesalaia/config/config_trocr.ini"
opts = Options(cfgfile)

In [ ]:
print(opts.installed_pckgs)
configuration_ok(opts, "cellrecognize")


In [ ]:
infolder ="/scratch/project_2005488/Muutto/Official/test-set/printed/images/man-ds-test2-all-printed"
infolder= "/scratch/project_2005488/Muutto/Official/development-set/printed/images/man-ds-test1-all-printed"

infolder = "/scratch/project_2005488/Muutto/Official/test-set/handdrawn/images/man-ds-test2-all-handdrawn"
infolder = "/scratch/project_2005488/Muutto/Official/development-set/handdrawn/images/man-ds-test1-all-handdrawn"


In [ ]:
infolder = "/scratch/project_2005488/Muutto/muuttaneet_1887-1891_ap"
inpage = "pageText"
dataset_files = [[infolder, os.path.join(infolder, inpage)]]
dataset1 = OCRDatasetInstanceSeg(dataset_files, {})
print(dataset1.__len__())

infolder = "/scratch/project_2005488/Muutto/muuttaneet_1887-1891_ap"
inpage = "pageText2"
dataset_files = [[infolder, os.path.join(infolder, inpage)]]
dataset2 = OCRDatasetInstanceSeg(dataset_files, {})
print(dataset2.__len__())


In [ ]:
infolder = "/scratch/project_2005488/Muutto/Official/test-set/handdrawn/images/man-ds-test2-all-handdrawn"
inpage = "pageText"
dataset_files = [[infolder, os.path.join(infolder, inpage)]]
dataset1 = OCRDatasetInstanceSeg(dataset_files, {})
print(dataset.__len__())

infolder = "/scratch/project_2005488/Muutto/Official/test-set/handdrawn/images/man-ds-test2-all-handdrawn"
inpage = "pageText"
dataset_files = [[infolder, os.path.join(infolder, inpage)]]
dataset1 = OCRDatasetInstanceSeg(dataset_files, {})
print(dataset.__len__())

In [ ]:
infolder = "/scratch/project_2005072/moving_records_htr/temp_for_pipeline//myrskyla_fold_9/images/myrskyla/muuttaneet_1774-1780_tk820"
infolder = "/scratch/project_2005072/moving_records_htr/temp_for_pipeline/myrskyla_fold_9/images/myrskyla/muuttaneet_1844-1889_ap_s"
inpage = "pageText"
dataset_files = [[infolder, os.path.join(infolder, inpage)]]
dataset1 = OCRDatasetInstanceSeg(dataset_files, {})
print(dataset1.__len__())


In [ ]:
idx = 0
img = cv2.imread(dataset1.__getfullname__(idx))
print(dataset1.__getfullname__(idx))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
page1 = dataset1.__getXMLitem__(idx)
print(len(page1['regions']))
plt.figure(figsize=(30, 20))
plt.imshow(img)
plt.show()

In [ ]:

for i in range(10):
    print(f"Page 1: {get_text(page1,2 , i)}")
    print()

In [ ]:
page1 = dataset1.__getXMLitem__(idx)
page2 = dataset2.__getXMLitem__(idx)
for i in range(10,20):
    print(f"Page 1: {get_text(page1, 5, i)}")
    print(f"Page 2: {get_text(page2, 5, i)}")
    print()


In [ ]:
def get_text(page, tblnr, rownr):
    text = ""
    for i, reg in enumerate(page['regions']):
        if i == tblnr:
            if 'table' in reg:
                for cell in reg['table']:
                    if 'lines' in cell:
                        for line in cell['lines']:
                            if rownr == cell['row']:
                                if isinstance(line['Text'], str):
                                    text = text + line['Text'] + "|"
                                elif isinstance(line['Text'], bytes):
                                    text = text + line['Text'].decode("utf-8") + "|"

    return text

In [ ]:
page = dataset.__getXMLitem__(idx)
get_text(page, 5, 0)

In [ ]:
same = 0
diff = 0
for idx in range(dataset.__len__()):
    page = dataset.__getXMLitem__(idx)
    page2 = dataset2.__getXMLitem__(idx)
    for i, reg in enumerate(page['regions']):
        if 'table' in reg:
            for r in range(reg['rows']):
                t1 = get_text(page, i, r)
                t2 = get_text(page2, i, r)
                if t1 == t2:
                    same += 1
                else:
                    print(t1)
                    print(t2)
                    diff += 1
    
    if idx > 3: break
print(same, diff)
print(same*100/(same+diff),diff*100/(same+diff))

In [ ]:
def get_polygons_of_nth_table(page, ptype, ntable):
    polygons = []
    reg = page['regions'][n]
    if ptype == "table":
       polygons.append(reg['polygon'])
    if 'table' in reg:
        for cell in reg['table']:
            if ptype == "cell":
                polygons.append(cell['polygon'])
            if 'lines' in cell:
                for line in cell['lines']:
                    if ptype == "line":
                        polygons.append(line['Textline'])
    return polygons

In [ ]:
vizpage = "viz"
for idx in range(dataset.__len__()):
    infile = dataset.__getfullname__(idx)
    print(infile)
    fname = os.path.basename(infile)
    vizdir = os.path.join(os.path.dirname(infile), vizpage)
    if not os.path.exists(vizdir):
        os.makedirs(vizdir)
    basename, _ = os.path.splitext(fname)
    vizfile = os.path.join(vizdir, basename + ".jpg")

    image = cv2.imread(infile)
    page = dataset.__getXMLitem__(idx)
    polygons = get_polygons(page, "cell")
    n_image = draw_polygons_on_image(image, polygons)
    plt.imshow(n_image)
    plt.title(vizfile)
    plt.show()
    if idx > 20: break
#    cv2.imwrite(vizfile, n_image)

In [ ]:
vizpage = "viz"
for idx in range(dataset.__len__()):
    infile = dataset.__getfullname__(idx)
    print(infile)
    fname = os.path.basename(infile)
    vizdir = os.path.join(os.path.dirname(infile), vizpage)
    if not os.path.exists(vizdir):
        os.makedirs(vizdir)
    basename, _ = os.path.splitext(fname)
    vizfile = os.path.join(vizdir, basename + ".jpg")

    image = cv2.imread(infile)
    page = dataset.__getXMLitem__(idx)
    for n in range(len(page['regions'])):
        polygons = get_polygons_of_nth_table(page, "cell",n)
        n_image = draw_polygons_on_image(image, polygons)
        plt.imshow(n_image)
        plt.title(vizfile)
        plt.show()
    if idx > 5: break
#    cv2.imwrite(vizfile, n_image)

In [ ]:
for idx in range(dataset.__len__()):
    img = cv2.imread(dataset.__getfullname__(idx))
   # print(img.shape)
    _, size = dataset.__getsize__(idx)
    print(f"Image: {img.shape} and from XML: {size}")

In [ ]:
from text_recognition.line2text import TRline2Text, text_recognition

In [ ]:
from model.inference import load_text_recognize_model 
load_text_recognize_model(opts, opts.device)

In [ ]:
import torch
import torch.nn.functional as F

def HTR_with_confidence(image, model, processor,  device):
    # Preprocess the batch
    pixel_values = processor(image, return_tensors='pt').pixel_values.to(device)
    
    # Generate output token ids and also return the logits
    with torch.no_grad():
        outputs = model.generate(pixel_values, return_dict_in_generate=True, output_scores=True)

    # Get generated token IDs
    generated_ids = outputs.sequences
    
    # Get logits (or the scores) from the output
    token_scores = outputs.scores  # List of logits for each generated token
    
    # Decode the generated text
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)

    # Calculate confidence scores
    confidence_scores = []
    for i, generated_id_seq in enumerate(generated_ids):
        token_confidences = []
        for j, token_id in enumerate(generated_id_seq):
            # Get the softmax over logits to convert to probabilities
            probs = F.softmax(token_scores[j], dim=-1)
            # Get the confidence for the predicted token (highest probability)
            token_confidence = probs[0, token_id].item()  # Get the probability of the predicted token
            token_confidences.append(token_confidence)
        confidence_scores.append(token_confidences)

    return generated_text, confidence_scores


In [ ]:
def htr(image, processor, model, device):
    """
    :param image: PIL Image.
    :param processor: Huggingface OCR processor.
    :param model: Huggingface OCR model.

    Returns:
        generated_text: the OCR'd text string.
    """
    # We can directly perform OCR on cropped images.
    pixel_values = processor(image, return_tensors='pt').pixel_values.to(device)
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_text

In [ ]:
idx = 7
page = dataset.__getXMLitem__(idx)
image = cv2.imread(dataset.__getfullname__(idx))
c_image = crop_polygon_from_image(image, page['regions'][0]['polygon'])
plt.imshow(c_image)
plt.show()
extracted_text, confidence = HTR_with_confidence(c_image, opts.text_recognize_model, opts.text_recognize_processor, opts.device)
print(extracted_text)
print(confidence)

In [ ]:
infolder = "/scratch/project_2005488/Muutto/sample10-all-printed" 
executeTask = "cell"
inpage = "pageTbl"
outpage = "pageCell"
cfgfile = "/users/vesalaia/config/config_table_line.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)

In [ ]:
executeTask = "cellrecognize"
inpage = "pageCell"
outpage = "pageText"
cfgfile = "/users/vesalaia/config/config_trocr.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)

In [ ]:
infolder = "/scratch/project_2005488/Muutto/man-ds-test1-all-printed" 
executeTask = "cell"
inpage = "pageTbl"
outpage = "pageCell"
cfgfile = "/users/vesalaia/config/config_table_line.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#




In [ ]:
infolder = "/scratch/project_2005488/Muutto/sample10-all-printed" 
executeTask = "cell"
inpage = "pageTbl"
outpage = "pageCell"
cfgfile = "/users/vesalaia/config/config_table_line.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
print_time()

In [ ]:
infolder = "/scratch/project_2005488/Muutto/sample5-all-printed" 

executeTask = "init"
inpage = ""
outpage = "page"
cfgfile = "/users/vesalaia/config/config_table_275.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)

executeTask = "table"
inpage = "page"
outpage = "pageTbl2"
cfgfile = "/users/vesalaia/config/config_table_275.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#

In [ ]:
infolder = "/scratch/project_2005488/Muutto/sample6-all-printed" 

executeTask = "init"
inpage = ""
outpage = "page"
cfgfile = "/users/vesalaia/config/config_table_275.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)

executeTask = "table"
inpage = "page"
outpage = "pageTbl2"
cfgfile = "/users/vesalaia/config/config_table_275.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#

In [ ]:
infolder = "/scratch/project_2005488/Muutto/sample7-all-printed" 

executeTask = "init"
inpage = ""
outpage = "page"
cfgfile = "/users/vesalaia/config/config_table_275.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)

executeTask = "table"
inpage = "page"
outpage = "pageTbl2"
cfgfile = "/users/vesalaia/config/config_table_275.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#

In [ ]:
infolder = "/scratch/project_2005488/Muutto/sample8-all-printed" 

executeTask = "init"
inpage = ""
outpage = "page"
cfgfile = "/users/vesalaia/config/config_table_275.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)

executeTask = "table"
inpage = "page"
outpage = "pageTbl2"
cfgfile = "/users/vesalaia/config/config_table_275.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#

In [ ]:
print_time()
infolder = "/scratch/project_2005488/Muutto/sample10-all-printed" 

executeTask = "init"
inpage = ""
outpage = "page"
cfgfile = "/users/vesalaia/config/config_table_275.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)

executeTask = "table"
inpage = "page"
outpage = "pageTbl2"
cfgfile = "/users/vesalaia/config/config_table_275.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
executeTask = "table"
inpage = "page"
outpage = "pageTbl3"
cfgfile = "/users/vesalaia/config/config_table_373.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#

print_time()
executeTask = "table"
inpage = "page"
outpage = "pageTbl4"
cfgfile = "/users/vesalaia/config/config_table_1100.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
print_time()
executeTask = "table"
inpage = "page"
outpage = "pageTbl5"
cfgfile = "/users/vesalaia/config/config_table_1492.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)



In [ ]:
print_time()
executeTask = "init"
infolder = "/scratch/project_2005488/Muutto/skewed" 
inpage = ""
outpage = "page"
cfgfile = "/users/vesalaia/config/config_table_1492.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
print_time()
executeTask = "table"
inpage = "page"
outpage = "pageTbl"
cfgfile = "/users/vesalaia/config/config_table_1492.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
print_time()
executeTask = "cell"
inpage = "pageTbl"
outpage = "pageCell"
cfgfile = "/users/vesalaia/config/config_table_1492.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
print_time()

executeTask = "cellrecognize"
inpage = "pageCell"
outpage = "pageText"
cfgfile = "/users/vesalaia/config/config_trocr.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
print_time()

In [ ]:
print_time()
executeTask = "init"
infolder = "/scratch/project_2005488/Muutto/deskewed" 
inpage = ""
outpage = "page"
cfgfile = "/users/vesalaia/config/config_table_1492.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
print_time()
executeTask = "table"
inpage = "page"
outpage = "pageTbl"
cfgfile = "/users/vesalaia/config/config_table_1492.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
print_time()
executeTask = "cell"
inpage = "pageTbl"
outpage = "pageCell"
cfgfile = "/users/vesalaia/config/config_table_1492.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
print_time()

executeTask = "cellrecognize"
inpage = "pageCell"
outpage = "pageText"
cfgfile = "/users/vesalaia/config/config_trocr.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
print_time()

In [ ]:
print_time()
executeTask = "init"
infolder = "/scratch/project_2005488/Muutto/sample9-all-printed" 
inpage = ""
outpage = "page2"
cfgfile = "/users/vesalaia/config/config_table.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
print_time()
executeTask = "table"
inpage = "page2"
outpage = "pageTbl"
cfgfile = "/users/vesalaia/config/config_table.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
#print_time()
#executeTask = "cell"
#inpage = "pageTbl"
#outpage = "pageCell"
#cfgfile = "/users/vesalaia/config/config_table.ini"
#processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
#print_time()

#executeTask = "cellrecognize"
#inpage = "pageCell"
#outpage = "pageText"
#cfgfile = "/users/vesalaia/config/config_trocr.ini"
#processTask(cfgfile, executeTask, infolder, inpage, outpage)
#rint_time()

In [ ]:
print_time()
executeTask = "init"
infolder = "/scratch/project_2005488/Muutto/sample6-all-printed" 
inpage = ""
outpage = "page"
cfgfile = "/users/vesalaia/config/config_table.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
print_time()
executeTask = "table"
inpage = "page"
outpage = "pageTbl"
cfgfile = "/users/vesalaia/config/config_table.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
print_time()
executeTask = "cell"
inpage = "pageTbl"
outpage = "pageCell"
cfgfile = "/users/vesalaia/config/config_table.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
print_time()

executeTask = "cellrecognize"
inpage = "pageCell"
outpage = "pageText"
cfgfile = "/users/vesalaia/config/config_trocr.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
print_time()

In [ ]:
print_time()
executeTask = "init"
infolder = "/scratch/project_2005488/Muutto/sample7-all-printed" 
inpage = ""
outpage = "page"
cfgfile = "/users/vesalaia/config/config_table.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
print_time()
executeTask = "table"
inpage = "page"
outpage = "pageTbl"
cfgfile = "/users/vesalaia/config/config_table.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
print_time()
executeTask = "cell"
inpage = "pageTbl"
outpage = "pageCell"
cfgfile = "/users/vesalaia/config/config_table.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
print_time()

executeTask = "cellrecognize"
inpage = "pageCell"
outpage = "pageText"
cfgfile = "/users/vesalaia/config/config_trocr.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
print_time()

In [ ]:
executeTask = "init"
infolder = "/scratch/project_2005488/Muutto/sample9-all-printed" 
inpage = ""
outpage = "pageTest"
cfgfile = "/users/vesalaia/config/config_table.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
executeTask = "table"
infolder = "/scratch/project_2005488/Muutto/sample9-all-printed" 
inpage = "pageTest"
outpage = "pageTbl"
cfgfile = "/users/vesalaia/config/config_table.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)


In [ ]:
print_time()
executeTask = "cellrecognize"
infolder = "/scratch/project_2005488/Muutto/sample3-all-printed" 
inpage = "pageCell"
outpage = "pageText2"
cfgfile = "/users/vesalaia/config/config_trocr.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
print_time()

In [ ]:
executeTask = "cellrecognize"
infolder = "/scratch/project_2005488/Muutto/Test"  
inpage = "pageCell"
outpage = "pageText"
cfgfile = "/users/vesalaia/config/config_trocr.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)

In [ ]:
print_time()
executeTask = "cellrecognize"
infolder = "/scratch/project_2005488/Muutto/Test" 
inpage = "pageCell"
outpage = "pageText"
cfgfile = "/users/vesalaia/config/config_trocr.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
print_time()

In [ ]:
print_time()
executeTask = "init"
infolder = "/scratch/project_2005488/Muutto/debug" 
inpage = ""
outpage = "page"
cfgfile = "/users/vesalaia/config/config_table.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
print_time()
executeTask = "table"
inpage = "page"
outpage = "pageTbl"
cfgfile = "/users/vesalaia/config/config_table.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
print_time()
executeTask = "cell"
inpage = "pageTbl"
outpage = "pageCell"
cfgfile = "/users/vesalaia/config/config_table.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
#
print_time()



In [ ]:
executeTask = "cellrecognize"
infolder = "/scratch/project_2005488/Muutto/debug" 
inpage = "pageCell"
outpage = "pageText"
cfgfile = "/users/vesalaia/config/config_trocr.ini"
processTask(cfgfile, executeTask, infolder, inpage, outpage)
print_time()

In [ ]:
inpage = "pageText"
dataset_files = [[infolder, os.path.join(infolder, inpage)]]
dataset = OCRDatasetInstanceSeg(dataset_files, {})

In [ ]:
page = dataset.__getXMLitem__(0)

In [ ]:
page